<a href="https://colab.research.google.com/github/sunflowcoco/SS_Classification/blob/master/CNN_SS_800_2way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
import sys
from keras.datasets import mnist

from keras.models import Sequential

from keras.layers import Input, Dense, Dropout, Flatten, Concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop  
from keras import backend as K

import numpy as np  
import os
import time

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

np.random.seed(1337)
from keras.utils import np_utils  
from keras.models import Model

#DNN setting
batch_size = 256
num_classes = 6
epochs = 100




###############################################################################
#If equall to 'yes', then load from Bin file
FAST_LOAD = 'yes'
#postfix
FILE_TYPE = '21_40d_62851r_122010'#'756d_less'#'1134d'

#Feature column
COLUMN = 800
#K-Fold
K = 10
k = 1
# if len(sys.argv)>=2:
# 	k = int(sys.argv[1])
# if len(sys.argv)>=3:
# 	K = int(sys.argv[2])	
  
path = '/content/drive/SS/'
	
#Log setting
LOG_FLAG = 1
log_str = ''
# logfilename = '../log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'
logfilename = path+ '/log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'


log_str = log_str + 'Start: '+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+'\r\n'
	
# src_file = '../data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file = '../data/sample_combinnativeflat21_40d_134685r_800.bin'

# src_file =path + 'data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file=path + 'data/sample_combinnativeflat21_40d_134685r_800.bin'


src_file =path + 'data/sample_combinnativeflat21_40d_134685r_12522.txt'
bin_file=path + 'data/sample_combinnativeflat21_40d_104685r_800.bin'

sample = []
if FAST_LOAD=='yes':
	sample = np.fromfile(bin_file, dtype=np.double)
	sample = sample.reshape(-1, COLUMN+1)
else:
	orig = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,40:840].astype(np.double).reshape(-1, COLUMN)
	label = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,-1].astype(np.double).reshape(-1, 1)-2
	
	print(orig.shape)
	print(label.shape)
	
	sample = np.hstack((orig, label))
	np.random.shuffle(sample)
	sample.tofile(bin_file)

	
gridLen = int(len(sample)*1.0/K)
s_index = (k-1)*gridLen
e_index = k*gridLen

testX = sample[s_index:e_index, :-1]
testY = sample[s_index:e_index, -1]

if k==1:
	trainX = sample[e_index:, :-1]
	trainY = sample[e_index:, -1]
elif k==K:
	trainX = sample[0:s_index, :-1]
	trainY = sample[0:s_index, -1]
else:	
	trainX = np.vstack((sample[0:s_index, :-1], sample[e_index:, :-1]))
	trainY = np.hstack((sample[0:s_index, -1], sample[e_index:, -1]))

trainYBackup = trainY
testYBackup = testY

print ('Train Size:'+str(len(trainY)))
print ('Test Size:'+str(len(testY)))



Using TensorFlow backend.


Train Size:94217
Test Size:10468


模型

In [0]:
###############################################################################

#save loss acc
class LossHistory(keras.callbacks.Callback):
	def on_train_begin(self, logs={}):
		self.losses = {'batch':[], 'epoch':[]}
		self.accuracy = {'batch':[], 'epoch':[]}
		self.val_loss = {'batch':[], 'epoch':[]}
		self.val_acc = {'batch':[], 'epoch':[]}

	def on_batch_end(self, batch, logs={}):
		self.losses['batch'].append(logs.get('loss'))
		self.accuracy['batch'].append(logs.get('acc'))
		self.val_loss['batch'].append(logs.get('val_loss'))
		self.val_acc['batch'].append(logs.get('val_acc'))

	def on_epoch_end(self, batch, logs={}):
		self.losses['epoch'].append(logs.get('loss'))
		self.accuracy['epoch'].append(logs.get('acc'))
		self.val_loss['epoch'].append(logs.get('val_loss'))
		self.val_acc['epoch'].append(logs.get('val_acc'))

	def getHistory(self):
		epoch_losses_str = ','.join(str(i) for i in self.losses['epoch'])
		epoch_accuracy_str = ','.join(str(i) for i in self.accuracy['epoch'])
		epoch_val_loss_str = ','.join(str(i) for i in self.val_loss['epoch'])
		epoch_val_acc_str = ','.join(str(i) for i in self.val_acc['epoch'])
		
		batch_losses_str = ','.join(str(i) for i in self.losses['batch'])
		batch_accuracy_str = ','.join(str(i) for i in self.accuracy['batch'])
		#batch_val_loss_str = ','.join(str(i) for i in self.val_loss['batch'])
		#batch_val_acc_str = ','.join(str(i) for i in self.val_acc['batch'])
		
		rt_str = ''
		rt_str = rt_str + 'epoch_losses:' + epoch_losses_str + '\r\n'
		rt_str = rt_str + 'epoch_accuracy:' + epoch_accuracy_str + '\r\n'
		rt_str = rt_str + 'epoch_val_loss:' + epoch_val_loss_str + '\r\n'
		rt_str = rt_str + 'epoch_val_acc:' + epoch_val_acc_str + '\r\n'
		
		rt_str = rt_str + 'batch_losses:' + batch_losses_str + '\r\n'
		rt_str = rt_str + 'batch_accuracy:' + batch_accuracy_str + '\r\n'
		#rt_str = rt_str + 'batch_val_loss:' + batch_val_loss_str + '\r\n'
		#rt_str = rt_str + 'batch_val_acc:' + batch_val_acc_str + '\r\n'
		
		return rt_str
		
###############################################################################
#history = LossHistory()

trainX=trainX*255
testX=testX*255

trainX1 = trainX[:, :320]
trainX2 = trainX[:, 320:]
# trainX1 = trainX1.reshape(-1, 20, 16, 1)
# trainX2 = trainX2.reshape(-1, 20, 24, 1)

trainX1 = trainX1.reshape(-1, 8, 40, 1)
trainX2 = trainX2.reshape(-1, 12, 40, 1)

testX1 = testX[:, :320]
testX2 = testX[:, 320:]
# testX1 = testX1.reshape(-1, 20, 16, 1)
# testX2 = testX2.reshape(-1, 20, 24, 1)
testX1 = testX1.reshape(-1, 8, 40, 1)
testX2 = testX2.reshape(-1, 12, 40, 1)

trainY=np_utils.to_categorical(trainY, num_classes)
testY=np_utils.to_categorical(testY, num_classes)


del trainX
del testX
del sample



模型

In [3]:
from keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_800.h5',monitor='val_loss',verbose=1,save_best_only=True)
# checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_800.h5',monitor='val_loss',verbose=1,save_best_only=False)
checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_800.h5',monitor='val_acc',verbose=1,save_best_only=True)

input_data1 = Input(shape=(8, 40, 1))
conv1 = Conv2D(32, kernel_size=(2, 2), activation='relu')(input_data1)
#conv1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv1 = Conv2D(64, (2, 2), activation='relu')(conv1)
conv1 = MaxPooling2D(pool_size=(2, 2))(conv1)
flat1 = Flatten()(conv1)

input_data2 = Input(shape=(12, 40, 1))
conv2 = Conv2D(32, kernel_size=(2, 2), activation='relu')(input_data2)
#conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv2 = Conv2D(64, (2, 2), activation='relu')(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat2 = Flatten()(conv2)

concatenated_tensor = Concatenate(axis=1)([flat1, flat2])

conv_output = Dense(128, activation='relu')(concatenated_tensor)
full = Dense(64, activation='relu')(conv_output)
full = Dense(32, activation='relu')(full)
full = Dense(16, activation='relu')(full)
full = Dropout(0.2)(full)
full_output = Dense(num_classes, activation='softmax')(full)

model = Model(inputs=[input_data1, input_data2], outputs=full_output)

rmsprop=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.00) 
model.compile(loss=keras.losses.categorical_crossentropy,
			  optimizer=rmsprop,#keras.optimizers.Adadelta(),
			  metrics=['accuracy'])

history=model.fit([trainX1, trainX2], trainY,
		  batch_size=batch_size,
		  epochs=epochs,
		  verbose=1,
		  validation_data=([testX1, testX2], testY),
		  callbacks=[checkpoint])


#model.save(os.path.basename(sys.argv[0])+'_model'+'_'+FILE_TYPE+'.h5')
test_rst = model.predict([testX1,testX2])

test_label = np.argmax(test_rst, axis=1).astype(np.int32)
#confusion_matrix = np.zeros((num_classes,num_classes))
testY = testYBackup.astype(np.int32)



####################评估训练效果####################		
print(history.history,file=open('history_800','w'))
cf=confusion_matrix(testY,test_label)
pre=precision_score(testY,test_label,average=None)
rec=recall_score(testY,test_label,average=None)
f1_s=f1_score(testY,test_label,average=None)

# print(cf,file=open('confusion_matrix_800','w'))
# print(pre,file=open('precision_800','w'))
# print(rec,file=open('recall_800','w'))
# print(f1_s,file=open('f1_score_800','w'))
print(cf)
print(pre)
print(rec)
print(f1_s)

acc=accuracy_score(testY,test_label)
print(acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 94217 samples, validate on 10468 samples
Epoch 1/100
94217/94217 [==============================] - 11s 121us/step - loss: 0.9893 - acc: 0.7342 - val_loss: 0.3496 - val_acc: 0.8860

Epoch 00001: val_acc improved from -inf to 0.88603, saving model to bestmodel_yyw_800.h5
Epoch 2/100
94217/94217 [==============================] - 9s 98us/step - loss: 0.3160 - acc: 0.8925 - val_loss: 0.2749 - val_acc: 0.9030

Epoch 00002: val_acc improved from 0.88603 to 0.90304, saving model to bestmodel_yyw_800.h5
Epoch 3/100
94217/94217 [==============================] - 9s 98us/step - loss: 0.2410 - acc: 0.9194 - val_loss: 0.2168 - val_acc: 0.9292

Epoch 00003: val_acc improved from 0.90304 to 0.92921, saving model to bestmodel_yyw_800.h5
Epoch 4/100
94217/94217